In [ ]:
# Cell 1: Install required Python packages
# This cell installs all necessary libraries for the healthcare dashboard.
# SQLite3 is built-in, but we install others for data handling, visualization, and Kaggle API.
!pip install pandas numpy matplotlib seaborn plotly ipywidgets kaggle
# Note: Ensure Kaggle API key is set up in Colab (upload kaggle.json or use secrets)

In [ ]:
# Cell 2: Import all necessary libraries
# Importing standard libraries for database, data manipulation, and visualization.
import sqlite3  # For SQLite database operations
import pandas as pd  # For data manipulation and DataFrames
import numpy as np  # For numerical operations and random data generation
import matplotlib.pyplot as plt  # For basic plotting
import seaborn as sns  # For statistical visualizations
import plotly.express as px  # For interactive plots
import plotly.graph_objects as go  # For advanced plotly figures
import ipywidgets as widgets  # For interactive widgets
from IPython.display import display, clear_output  # For displaying widgets and outputs
from datetime import datetime, timedelta  # For date handling
from kaggle.api.kaggle_api_extended import KaggleApi  # For Kaggle API to download datasets
import random  # For random data generation
import os  # For file operations

In [ ]:
# Cell 3: Database Schema Creation
# This function creates the relational schema for the healthcare database.
# Tables: Patients, Doctors, VitalsLog, Medications with foreign key relationships.

def create_database_schema(db_path='healthcare.db'):
    """
    Creates the SQLite database schema for the Healthcare Patient Monitoring Hub.
    
    Args:
        db_path (str): Path to the SQLite database file.
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Create Patients table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Patients (
                patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                age INTEGER,
                gender TEXT,
                admission_date TEXT,
                doctor_id INTEGER,
                FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id)
            )
        ''')
        
        # Create Doctors table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Doctors (
                doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                specialty TEXT,
                shift_start TEXT,
                shift_end TEXT
            )
        ''')
        
        # Create VitalsLog table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS VitalsLog (
                vitals_id INTEGER PRIMARY KEY AUTOINCREMENT,
                patient_id INTEGER,
                timestamp TEXT,
                heart_rate INTEGER,
                blood_pressure TEXT,
                temperature REAL,
                FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
            )
        ''')
        
        # Create Medications table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Medications (
                med_id INTEGER PRIMARY KEY AUTOINCREMENT,
                patient_id INTEGER,
                medication_name TEXT,
                dosage TEXT,
                frequency TEXT,
                administered_date TEXT,
                FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
            )
        ''')
        
        conn.commit()
        print("Database schema created successfully.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

# Call the function to create the schema
create_database_schema()

In [ ]:
# Cell 4: Batch Data Simulation
# This section generates and inserts mock data using Kaggle dataset and synthetic generation.

def download_kaggle_dataset(dataset_slug='ronitf/heart-disease-uci', download_path='.'):
    """
    Downloads a dataset from Kaggle using the API.
    
    Args:
        dataset_slug (str): Kaggle dataset identifier.
        download_path (str): Path to download the dataset.
    """
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset_slug, path=download_path, unzip=True)
    print(f"Downloaded {dataset_slug} to {download_path}")

def generate_mock_data(num_patients=1000, num_doctors=50, num_vitals_per_patient=10, num_meds_per_patient=5):
    """
    Generates synthetic data for the database tables.
    
    Args:
        num_patients (int): Number of patients to generate.
        num_doctors (int): Number of doctors.
        num_vitals_per_patient (int): Vitals logs per patient.
        num_meds_per_patient (int): Medications per patient.
    
    Returns:
        dict: Dictionary of DataFrames for each table.
    """
    # Generate Doctors
    doctors = pd.DataFrame({
        'name': [f'Dr. {random.choice(["Smith", "Johnson", "Williams", "Brown", "Jones"])} {i}' for i in range(num_doctors)],
        'specialty': np.random.choice(['Cardiology', 'Neurology', 'Oncology', 'Pediatrics', 'General'], num_doctors),
        'shift_start': np.random.choice(['08:00', '12:00', '16:00'], num_doctors),
        'shift_end': np.random.choice(['16:00', '20:00', '00:00'], num_doctors)
    })
    
    # Generate Patients
    patients = pd.DataFrame({
        'name': [f'Patient {i}' for i in range(num_patients)],
        'age': np.random.randint(18, 90, num_patients),
        'gender': np.random.choice(['Male', 'Female'], num_patients),
        'admission_date': [(datetime.now() - timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d') for _ in range(num_patients)],
        'doctor_id': np.random.randint(1, num_doctors+1, num_patients)
    })
    
    # Generate VitalsLog
    vitals = []
    for pid in range(1, num_patients+1):
        for _ in range(num_vitals_per_patient):
            vitals.append({
                'patient_id': pid,
                'timestamp': (datetime.now() - timedelta(hours=random.randint(0, 720))).strftime('%Y-%m-%d %H:%M:%S'),
                'heart_rate': np.random.randint(60, 120),
                'blood_pressure': f'{np.random.randint(90, 140)}/{np.random.randint(60, 90)}',
                'temperature': round(np.random.uniform(36.5, 38.5), 1)
            })
    vitals_df = pd.DataFrame(vitals)
    
    # Generate Medications
    meds = []
    med_names = ['Aspirin', 'Ibuprofen', 'Paracetamol', 'Insulin', 'Antibiotics']
    for pid in range(1, num_patients+1):
        for _ in range(num_meds_per_patient):
            meds.append({
                'patient_id': pid,
                'medication_name': random.choice(med_names),
                'dosage': f'{random.randint(1, 500)}mg',
                'frequency': random.choice(['Once daily', 'Twice daily', 'Three times daily']),
                'administered_date': (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d')
            })
    meds_df = pd.DataFrame(meds)
    
    return {'doctors': doctors, 'patients': patients, 'vitals': vitals_df, 'medications': meds_df}

def insert_data_to_db(data_dict, db_path='healthcare.db'):
    """
    Inserts generated data into the database.
    
    Args:
        data_dict (dict): Dictionary of DataFrames.
        db_path (str): Database path.
    """
    conn = sqlite3.connect(db_path)
    
    data_dict['doctors'].to_sql('Doctors', conn, if_exists='append', index=False)
    data_dict['patients'].to_sql('Patients', conn, if_exists='append', index=False)
    data_dict['vitals'].to_sql('VitalsLog', conn, if_exists='append', index=False)
    data_dict['medications'].to_sql('Medications', conn, if_exists='append', index=False)
    
    conn.close()
    print("Data inserted successfully.")

# Download Kaggle data (optional, for reference)
# download_kaggle_dataset()

# Generate and insert mock data
mock_data = generate_mock_data()
insert_data_to_db(mock_data)

In [ ]:
# Cell 5: Advanced SQL Queries
# Functions to execute complex SQL queries for high-risk patients and busiest shifts.

def get_high_risk_patients(db_path='healthcare.db', threshold=100):
    """
    Identifies high-risk patients based on average heart rate over last 3 checks.
    
    Args:
        db_path (str): Database path.
        threshold (int): Heart rate threshold.
    
    Returns:
        pd.DataFrame: High-risk patients.
    """
    conn = sqlite3.connect(db_path)
    query = '''
        SELECT p.patient_id, p.name, AVG(v.heart_rate) as avg_heart_rate
        FROM Patients p
        JOIN VitalsLog v ON p.patient_id = v.patient_id
        WHERE v.timestamp IN (
            SELECT timestamp FROM VitalsLog WHERE patient_id = p.patient_id ORDER BY timestamp DESC LIMIT 3
        )
        GROUP BY p.patient_id, p.name
        HAVING AVG(v.heart_rate) > ?
    '''
    df = pd.read_sql_query(query, conn, params=(threshold,))
    conn.close()
    return df

def get_busiest_shifts(db_path='healthcare.db'):
    """
    Calculates busiest shift hours for doctors based on vitals checks.
    
    Args:
        db_path (str): Database path.
    
    Returns:
        pd.DataFrame: Busiest hours.
    """
    conn = sqlite3.connect(db_path)
    query = '''
        SELECT strftime('%H', v.timestamp) as hour, COUNT(*) as checks
        FROM VitalsLog v
        JOIN Patients p ON v.patient_id = p.patient_id
        JOIN Doctors d ON p.doctor_id = d.doctor_id
        GROUP BY hour
        ORDER BY checks DESC
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Example usage
high_risk = get_high_risk_patients()
print("High-risk patients:")
print(high_risk.head())

busiest = get_busiest_shifts()
print("Busiest shift hours:")
print(busiest.head())

In [ ]:
# Cell 6: Interactive Dashboard
# Creates an interactive dashboard with widgets and visualizations.

def create_dashboard(db_path='healthcare.db'):
    """
    Builds the interactive dashboard.
    
    Args:
        db_path (str): Database path.
    """
    conn = sqlite3.connect(db_path)
    
    # Load data
    patients = pd.read_sql_query("SELECT * FROM Patients", conn)
    vitals = pd.read_sql_query("SELECT * FROM VitalsLog", conn)
    meds = pd.read_sql_query("SELECT * FROM Medications", conn)
    
    conn.close()
    
    # Widgets
    patient_dropdown = widgets.Dropdown(
        options=patients['name'].tolist(),
        description='Patient:',
        value=patients['name'].iloc[0]
    )
    
    output = widgets.Output()
    
    def update_dashboard(change):
        with output:
            clear_output(wait=True)
            selected_patient = patients[patients['name'] == patient_dropdown.value]['patient_id'].iloc[0]
            
            # Vitals trend
            patient_vitals = vitals[vitals['patient_id'] == selected_patient].sort_values('timestamp')
            fig1 = px.line(patient_vitals, x='timestamp', y='heart_rate', title='Heart Rate Over Time')
            fig1.show()
            
            # Medication usage
            med_counts = meds['medication_name'].value_counts()
            fig2 = px.bar(med_counts, title='Medication Usage Frequency')
            fig2.show()
    
    patient_dropdown.observe(update_dashboard, names='value')
    
    # Initial display
    update_dashboard(None)
    
    display(patient_dropdown, output)

# Run the dashboard
create_dashboard()